In [1]:
import pandas as pd
df = pd.read_csv('v8.6output.csv')
df

,Entity.LegalAddress.City,Entity.LegalAddress.Region,Entity.LegalAddress.PostalCode,Entity.HeadquartersAddress.City,Entity.HeadquartersAddress.Region,Entity.HeadquartersAddress.PostalCode,Entity.LegalAddress.City/Entity.LegalAddress.Region,Entity.LegalAddress.City/Entity.LegalAddress.PostalCode,Entity.LegalAddress.City/Entity.HeadquartersAddress.City,Entity.LegalAddress.City/Entity.HeadquartersAddress.Region,...,Entity.LegalAddress.Region/Entity.LegalAddress.PostalCode,Entity.LegalAddress.Region/Entity.HeadquartersAddress.City,Entity.LegalAddress.Region/Entity.HeadquartersAddress.Region,Entity.LegalAddress.Region/Entity.HeadquartersAddress.PostalCode,Entity.LegalAddress.PostalCode/Entity.HeadquartersAddress.City,Entity.LegalAddress.PostalCode/Entity.HeadquartersAddress.Region,Entity.LegalAddress.PostalCode/Entity.HeadquartersAddress.PostalCode,Entity.HeadquartersAddress.City/Entity.HeadquartersAddress.Region,Entity.HeadquartersAddress.City/Entity.HeadquartersAddress.PostalCode,Entity.HeadquartersAddress.Region/Entity.HeadquartersAddress.PostalCode
0,BOSTON,US-MA,02110,Boston,US-MA,02210,0.702805,0.436085,0.392924,0.255097,...,0.307488,0.284994,0.486177,0.133923,0.083143,0.054378,0.011166,0.594713,0.295168,0.176236
1,Wilmington,US-DE,19808,New York,US-NY,10106,0.822073,0.384763,0.120697,0.108123,...,0.318466,0.133935,0.126153,0.000493,0.067257,0.057407,0.000039,0.749300,0.002989,0.002307
2,MALVERN,US-PA,19355,Valley Forge,US-PA,19482,0.010966,0.846630,0.045873,0.007581,...,0.011365,0.000603,0.600256,0.000403,0.041602,0.007857,0.026065,0.029801,0.943380,0.028475
3,WILMINGTON,US-DE,19801,Greenwich,US-CT,06830,0.822073,0.530562,0.016864,0.017346,...,0.439650,0.018321,0.021003,0.018101,0.021916,0.012883,0.022688,0.327437,0.934107,0.307061
4,BOSTON,US-MA,02110,Chicago,US-IL,60606,0.702805,0.436085,0.000990,0.002739,...,0.307488,0.000691,0.001923,0.001618,0.001314,0.004630,0.003218,0.512780,0.336975,0.173440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Wilmington,US-DE,19801,Frisco,US-TX,75035,0.822073,0.530562,0.000031,0.018486,...,0.439650,0.000077,0.025254,0.000010,0.000031,0.010930,0.000002,0.007014,0.105769,0.000756
4996,Monroe,US-GA,30655,Largo,US-FL,33778,0.000389,0.113208,0.002695,0.000013,...,0.003440,0.000082,0.000066,0.000459,0.023810,0.000114,0.133333,0.004460,0.178571,0.000856
4997,Minnetonka,US-MN,55305,Minnetonka,US-MN,55305,0.016934,0.268680,0.223191,0.011223,...,0.007780,0.005053,0.638691,0.001860,0.047438,0.004487,0.157796,0.042034,0.430620,0.021356
4998,NEWPORT,US-RI,02840,NEWPORT,US-RI,02840,0.011166,0.301370,0.563378,0.005352,...,0.033898,0.008601,0.517361,0.024653,0.147296,0.015939,0.465455,0.009261,0.316456,0.027056


In [2]:
new_df = df[['Entity.LegalAddress.City', 'Entity.LegalAddress.Region', 'Entity.LegalAddress.PostalCode']].copy()
new_df['Entity.LegalAddress.Region'] = new_df['Entity.LegalAddress.Region'].str[3:]
new_df['row_values'] = new_df['Entity.LegalAddress.City'].map(str) + ' ' + new_df['Entity.LegalAddress.Region'].map(str) + ' ' + df['Entity.LegalAddress.PostalCode'].map(str)
new_df

,Entity.LegalAddress.City,Entity.LegalAddress.Region,Entity.LegalAddress.PostalCode,row_values
0,BOSTON,MA,02110,BOSTON MA 02110
1,Wilmington,DE,19808,Wilmington DE 19808
2,MALVERN,PA,19355,MALVERN PA 19355
3,WILMINGTON,DE,19801,WILMINGTON DE 19801
4,BOSTON,MA,02110,BOSTON MA 02110
...,...,...,...,...
4995,Wilmington,DE,19801,Wilmington DE 19801
4996,Monroe,GA,30655,Monroe GA 30655
4997,Minnetonka,MN,55305,Minnetonka MN 55305
4998,NEWPORT,RI,02840,NEWPORT RI 02840


In [23]:
row_values = new_df[['row_values']].copy()

import numpy as np
from numpy.random import choice
row_values['validity'] = np.random.choice([0,1], size=len(row_values), p=[.10,.90])
row_values['row_values'] = row_values['row_values'].str.lower()

row_values

,row_values,validity
0,boston ma 02110,1
1,wilmington de 19808,0
2,malvern pa 19355,0
3,wilmington de 19801,1
4,boston ma 02110,0
...,...,...
4995,wilmington de 19801,1
4996,monroe ga 30655,1
4997,minnetonka mn 55305,1
4998,newport ri 02840,0


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

vect = TfidfVectorizer(ngram_range=(2,3))
summaries = ' '.join(row_values['row_values'])
ngram_summaries = vect.build_analyzer()(summaries)

Counter(ngram_summaries).most_common(20)

[('wilmington de', 1680),
 ('de 19801', 863),
 ('wilmington de 19801', 856),
 ('de 19808', 668),
 ('wilmington de 19808', 662),
 ('dover de', 319),
 ('19801 wilmington', 315),
 ('de 19801 wilmington', 315),
 ('19801 wilmington de', 314),
 ('19808 wilmington', 243),
 ('19808 wilmington de', 242),
 ('de 19808 wilmington', 241),
 ('de 19904', 185),
 ('new york', 184),
 ('dover de 19904', 182),
 ('york ny', 180),
 ('new york ny', 180),
 ('boston ma', 166),
 ('de 19901', 137),
 ('dover de 19901', 132)]

In [25]:
from sklearn.model_selection import train_test_split
data = row_values.copy()

data = row_values.copy()
y = data['validity'].values
data.drop(['validity'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size= 0.2, random_state=42)
print("Train data:", X_train.shape, y_train.shape)
print("Test data:", X_test.shape, y_test.shape)

Train data: (4000, 1) (4000,)
Test data: (1000, 1) (1000,)


In [26]:
y_train = pd.Series(y_train).values
y_test = pd.Series(y_test).values

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer


title_text = X_train['row_values'].values

tfv = TfidfVectorizer(ngram_range=(2,4), max_features=2)
X_train = tfv.fit_transform(title_text).todense()
print(X_train.shape)

testing = X_test['row_values'].values

tfv = TfidfVectorizer(ngram_range=(2,3), max_features=2)
X_test = tfv.fit_transform(testing).todense()
print(X_test.shape)

(4000, 2)
(1000, 2)


In [28]:
from sklearn.metrics import confusion_matrix

In [29]:
from sklearn.neighbors import KNeighborsClassifier

knnclassifier = KNeighborsClassifier(n_neighbors=1000)
knnclassifier.fit(X_train,y_train)

y_pred = knnclassifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

accuracy = float(cm.diagonal().sum())/len(y_test)
print("\nAccuracy Of Row Validity Check with KNN : ", accuracy)


Accuracy Of Row Validity Check with KNN :  0.899


In [30]:
from sklearn.naive_bayes import GaussianNB

nbclassifier = GaussianNB()
nbclassifier.fit(X_train,y_train)

y_pred = nbclassifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

accuracy = float(cm.diagonal().sum())/len(y_test)
print("\nAccuracy Of Row Validity Check with NBayes : ", accuracy)


Accuracy Of Row Validity Check with NBayes :  0.899


In [31]:
from sklearn.neural_network import MLPClassifier

nnclassifier = MLPClassifier(solver='adam', alpha=5, max_iter=1000)
nnclassifier.fit(X_train, y_train)

y_pred = nnclassifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

accuracy = float(cm.diagonal().sum())/len(y_test)
print("\nAccuracy Of Row Validity Check with DNN : ", accuracy)


Accuracy Of Row Validity Check with DNN :  0.899


In [32]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

accuracy = float(cm.diagonal().sum())/len(y_test)
print("\nAccuracy Of Row Valididty Check with SVM : ", accuracy)


Accuracy Of Row Valididty Check with SVM :  0.899
